# Experimento sobre algoritmos de ordenação

##### Autor: Estevão de Carvalho Costa

In [81]:
import random
import time
import matplotlib
import numpy as np

## 1. Implementações

### 1.1. Heap sort

In [61]:
## Utility functions

def index_of_left_child(i):
    return 2*i
def index_of_right_child(i):
    return 2*i + 1

def heapify(heap_array, i, size):
    maximum = i
    while(i <= size):
        left = index_of_left_child(i)
        right = index_of_right_child(i)
        if(left <= size and heap_array[left] > heap_array[i]):
            maximum = left
        if(right <= size and heap_array[right] > heap_array[maximum]):
            maximum = right
        if(maximum != i):
            heap_array[maximum], heap_array[i] = heap_array[i], heap_array[maximum]
            i = maximum
        else:
            break

def build_heap(array, number_of_elements):
    array.insert(0, None)
    heap_array = array
    index_of_first_non_leaf_node = round(number_of_elements/2)
    for i in range(index_of_first_non_leaf_node, 0, -1):
        heapify(heap_array, i, number_of_elements)
        
def show_heap(heap_array):
    nodes_buffer = [heap_array[1]]
    current = 1
    while(len(nodes_buffer) > 0):
        for node in nodes_buffer:
            print(node, end=' ')
            nodes_buffer = nodes_buffer[1:]
            if(index_of_left_child(current) <= size):
                nodes_buffer.append(heap_array[index_of_left_child(current)])
            if(index_of_right_child(current) <= size):
                nodes_buffer.append(heap_array[index_of_right_child(current)])
            current += 1
        print()
        
## main heap sort function:

def heap_sort(array):
    number_of_elements = len(array)
    build_heap(array, number_of_elements)
    for i in range(number_of_elements, 1, -1):
        array[1], array[i] = array[i], array[1]
        number_of_elements -= 1
        heapify(array, 1, number_of_elements)
    array.pop(0)


In [62]:
array = [4,10,50,20,1,3,5]
heap_sort(array)
print(array)

[1, 3, 4, 5, 10, 20, 50]


### 1.2. Quick Sort

In [63]:
def partition(array, low, high):
    pivot = array[high]
    i = low - 1
    for j in range(low, high):
        if(array[j] <= pivot):
            i += 1
            array[i], array[j] = array[j], array[i]
    array[i+1], array[high] = array[high], array[i+1]
    return i + 1

def helper_quick_sort(array, low, high):
    if(high > low):
        middle_index = partition(array, low, high)    
        helper_quick_sort(array, low, middle_index - 1)
        helper_quick_sort(array, middle_index + 1, high)

def quick_sort(array):
    helper_quick_sort(array, 0, len(array) - 1)    
    

In [64]:
array = [78,5,578,8,6,4,5,7,8,5,4,3,56]
quick_sort(array)
print(array)

[3, 4, 4, 5, 5, 5, 6, 7, 8, 8, 56, 78, 578]


### 1.3. Counting Sort e Radix Sort

Os algoritmos Counting Sort e Radix Sort assumem que as chaves de `array` pertencem ao intervalo `[0, maximum_key_value]`

In [65]:
def counting_sort(array, maximum_key_value, key):
    countingLists = []
    for i in range(0, maximum_key_value + 1):
        countingLists.append([])
    n = len(array)
    sorted_array = []*n
    for i in range(0, n):
        countingLists[key(array[i])].append(array[i])
    for i in range(0, maximum_key_value + 1):
          sorted_array.extend(countingLists[i])
    return sorted_array

In [66]:
sorted_array = counting_sort([1,0,4,6,4,3,6,6], 6, lambda number : number)
print(sorted_array)

[0, 1, 3, 4, 4, 6, 6, 6]


In [67]:
def radix_sort(array, maximun_key_value, digits):
    sorted_array = array
    for i in range(0, digits):
        ith_less_significant_digit = lambda number: (number // 10**i) % 10
        sorted_array = counting_sort(sorted_array, maximun_key_value, ith_less_significant_digit)
    return sorted_array

In [68]:
array = [176,220,123,40,50,1,0,123,312,3,12,3,123,12,123,12,445,67,1233,6664]
sorted_array = radix_sort(array, 176, 4)
print(sorted_array)

[0, 1, 3, 3, 12, 12, 12, 40, 50, 67, 123, 123, 123, 123, 176, 220, 312, 445, 1233, 6664]


#### Nos experimentos a seguir, vamos plotar gráficos comparando em cada algoritmo implementado o **tempo de execução** em função do **tamanho do vetor de entrada**

## 2. Experimento 1

#### Deve-se ordenar vetores numéricos onde em média 90% dos elementos têm o mesmo valor. Os demais apresentem valores distintos e distribuídos de maneira uniforme o longo do vetor.

## 3. Experimento 2

#### Ordenar vetores onde não há elementos repetidos e chaves foram inseridas de maneira aleatória.

In [82]:
def __is_sorted(array):
    for i in range(1, len(array)):
        if(array[i-1] > array[i]):
            return False
    return True
        
def __get_unique_key(keys_already_chosen, maximum_key_value):
    candidate = random.randint(0, maximum_key_value)
    while(keys_already_chosen.get(candidate) != None):
        candidate = random.randint(0, maximum_key_value)
    return candidate
    
def generate_array_with_random_and_unique_keys(n, maximum_key_value = 1000):
    if(maximum_key_value < n):
        maximum_key_value = n + 100
    array = [None] * n
    keys_already_chosen = {}
    for i in range(n):
        new_random_key = __get_unique_key(keys_already_chosen, maximum_key_value)
        keys_already_chosen[new_random_key] = True
        array[i] = new_random_key
    return array

def execution_times_to_sort_random_arrays(maximun_array_size):
    execution_times = [None] * (maximun_array_size + 1)
    for n in range(1, maximun_array_size + 1, 10):
        random_array = generate_array_with_random_and_unique_keys(n)
        random_array_copy_1 = random_array.copy()
        random_array_copy_2 = random_array.copy()
        maximun_key_value = max(random_array)
        digits = len(str(maximun_array_size))
        execution_time = {}
        
        start_time = time.time()
        heap_sort(random_array)
        execution_time['heap_sort'] = time.time() - start_time
        if(not __is_sorted(random_array)):
            print("heap sort didn't sorted this array:")
            print(random_array)
        
        start_time = time.time()
        quick_sort(random_array_copy_1)
        execution_time['quick_sort'] = time.time() - start_time
        if(not __is_sorted(random_array_copy_1)):
            print("quick sort didn't sorted this array:")
            print(random_array_copy_1)
        
        start_time = time.time()
        sorted_array = radix_sort(random_array_copy_2, maximun_key_value, digits)
        execution_time['radix_sort'] = time.time() - start_time
        if(not __is_sorted(random_array_copy_2)):
            print("radix sort didn't sorted this array:")
            print(random_array_copy_2)
        
        execution_times[n] = execution_time
    
    return execution_times  
    


In [86]:
## TODO: debugar - descobrir por que radix não está ordenando estes vetores
execution_times = execution_times_to_sort_random_arrays(500)

radix sort didn't sorted this array:
[717, 789, 141, 923, 949, 50, 316, 927, 538, 418, 203]
radix sort didn't sorted this array:
[450, 105, 461, 778, 338, 707, 93, 347, 299, 950, 640, 190, 318, 286, 569, 823, 773, 8, 750, 932, 884]
radix sort didn't sorted this array:
[740, 559, 148, 360, 738, 232, 479, 632, 641, 841, 36, 159, 493, 437, 368, 650, 290, 373, 471, 165, 351, 704, 172, 674, 780, 97, 118, 679, 927, 745, 319]
radix sort didn't sorted this array:
[471, 917, 531, 784, 163, 755, 695, 281, 553, 125, 46, 191, 274, 925, 207, 962, 664, 241, 671, 496, 645, 564, 961, 346, 654, 358, 96, 29, 296, 195, 442, 279, 387, 999, 459, 237, 477, 909, 577, 193, 187]
radix sort didn't sorted this array:
[84, 490, 870, 903, 761, 658, 419, 480, 988, 83, 91, 633, 88, 21, 587, 417, 421, 646, 100, 779, 1, 317, 689, 727, 110, 351, 828, 669, 716, 734, 688, 898, 629, 370, 916, 877, 119, 361, 695, 909, 192, 871, 713, 117, 52, 852, 50, 219, 482, 231, 913]
radix sort didn't sorted this array:
[297, 183, 211, 

In [79]:
for n in range(len(execution_times)):
    if(execution_times[n] != None):
        print(f"size = {n}, results: {execution_times[n]}")

size = 1, results: {'heap_sort': 6.67572021484375e-06, 'quick_sort': 1.6689300537109375e-06, 'radix_sort': 2.5510787963867188e-05}
size = 11, results: {'heap_sort': 3.647804260253906e-05, 'quick_sort': 1.2874603271484375e-05, 'radix_sort': 0.00089263916015625}
size = 21, results: {'heap_sort': 8.225440979003906e-05, 'quick_sort': 3.62396240234375e-05, 'radix_sort': 0.0014469623565673828}
size = 31, results: {'heap_sort': 0.0001285076141357422, 'quick_sort': 5.4836273193359375e-05, 'radix_sort': 0.0008156299591064453}
size = 41, results: {'heap_sort': 0.0001709461212158203, 'quick_sort': 5.5789947509765625e-05, 'radix_sort': 0.0009884834289550781}
size = 51, results: {'heap_sort': 0.00029349327087402344, 'quick_sort': 0.0001418590545654297, 'radix_sort': 0.0008730888366699219}
size = 61, results: {'heap_sort': 0.0002887248992919922, 'quick_sort': 0.00010275840759277344, 'radix_sort': 0.0010843276977539062}
size = 71, results: {'heap_sort': 0.0003440380096435547, 'quick_sort': 0.00012731